# 🔧 Preprocesamiento de Texto
## Pipeline completo de limpieza y normalización

### Objetivos:
1. Implementar pipeline de preprocesamiento completo
2. Limpiar y normalizar el texto
3. Aplicar tokenización, eliminación de stopwords y lematización
4. Guardar datos preprocesados
5. Comparar texto original vs preprocesado


## 1. Importar librerías


In [ ]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Añadir src al path para importar módulos
sys.path.append(str(Path('../src').resolve()))

from data.preprocessing import TextPreprocessor

print("✅ Librerías importadas")


## 2. Cargar datos


In [ ]:
# Cargar dataset
data_path = Path('../data/raw/youtoxic_english_1000.csv')
df = pd.read_csv(data_path)

print(f"✅ Dataset cargado: {len(df)} filas, {len(df.columns)} columnas")
print(f"\n📋 Primeras filas:")
df.head()


## 3. Identificar columnas de texto y etiquetas


In [ ]:
# Identificar columna de texto
text_col = 'Text'  # Según el EDA, esta es la columna de texto
label_col = 'IsToxic'  # Columna principal de etiquetas

# Verificar que existen
assert text_col in df.columns, f"Columna '{text_col}' no encontrada"
assert label_col in df.columns, f"Columna '{label_col}' no encontrada"

print(f"✅ Columna de texto: '{text_col}'")
print(f"✅ Columna de etiquetas: '{label_col}'")

# Verificar valores nulos
print(f"\n🔍 Valores nulos en '{text_col}': {df[text_col].isnull().sum()}")
print(f"🔍 Valores nulos en '{label_col}': {df[label_col].isnull().sum()}")

# Eliminar filas con texto nulo
df = df.dropna(subset=[text_col])
print(f"\n✅ Dataset después de eliminar nulos: {len(df)} filas")


## 4. Inicializar preprocesador


In [ ]:
# Inicializar preprocesador (usa spaCy si está disponible)
preprocessor = TextPreprocessor(use_spacy=True)

print("✅ Preprocesador inicializado")
print(f"   Usando: {'spaCy' if preprocessor.use_spacy else 'NLTK'}")


## 5. Ejemplos de preprocesamiento


In [ ]:
# Mostrar ejemplos de preprocesamiento
print("="*80)
print("EJEMPLOS DE PREPROCESAMIENTO")
print("="*80)

sample_texts = df[text_col].sample(5, random_state=42).tolist()

for i, text in enumerate(sample_texts, 1):
    processed = preprocessor.preprocess_text(text, remove_stopwords=True)
    print(f"\n📝 Ejemplo {i}:")
    print(f"   Original:  {text[:150]}..." if len(text) > 150 else f"   Original:  {text}")
    print(f"   Procesado: {processed[:150]}..." if len(processed) > 150 else f"   Procesado: {processed}")


## 6. Aplicar preprocesamiento a todo el dataset


In [ ]:
# Aplicar preprocesamiento
print("="*80)
print("PREPROCESANDO DATASET COMPLETO")
print("="*80)
print("⏳ Esto puede tardar unos minutos...\n")

df_processed = preprocessor.preprocess_dataframe(
    df, 
    text_column=text_col,
    output_column='Text_processed',
    remove_stopwords=True,
    show_progress=True
)

print(f"\n✅ Preprocesamiento completado")
print(f"   Filas procesadas: {len(df_processed)}")


## 7. Análisis del preprocesamiento


In [ ]:
# Calcular estadísticas
df_processed['text_length_original'] = df_processed[text_col].str.len()
df_processed['text_length_processed'] = df_processed['Text_processed'].str.len()
df_processed['word_count_original'] = df_processed[text_col].str.split().str.len()
df_processed['word_count_processed'] = df_processed['Text_processed'].str.split().str.len()

print("="*80)
print("ESTADÍSTICAS DE PREPROCESAMIENTO")
print("="*80)

stats = pd.DataFrame({
    'Original': [
        df_processed['text_length_original'].mean(),
        df_processed['word_count_original'].mean(),
        df_processed['text_length_original'].min(),
        df_processed['text_length_original'].max()
    ],
    'Procesado': [
        df_processed['text_length_processed'].mean(),
        df_processed['word_count_processed'].mean(),
        df_processed['text_length_processed'].min(),
        df_processed['text_length_processed'].max()
    ]
}, index=['Longitud promedio (caracteres)', 'Palabras promedio', 'Longitud mínima', 'Longitud máxima'])

print("\n📊 Comparación:")
print(stats)

print(f"\n📉 Reducción promedio:")
print(f"   Caracteres: {((df_processed['text_length_original'].mean() - df_processed['text_length_processed'].mean()) / df_processed['text_length_original'].mean() * 100):.1f}%")
print(f"   Palabras: {((df_processed['word_count_original'].mean() - df_processed['word_count_processed'].mean()) / df_processed['word_count_original'].mean() * 100):.1f}%")


## 8. Visualización de resultados


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histograma de longitud original vs procesada
axes[0, 0].hist(df_processed['text_length_original'], bins=50, alpha=0.7, label='Original', color='#3498db')
axes[0, 0].hist(df_processed['text_length_processed'], bins=50, alpha=0.7, label='Procesado', color='#e74c3c')
axes[0, 0].set_xlabel('Longitud (caracteres)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Longitud de Texto', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Histograma de número de palabras
axes[0, 1].hist(df_processed['word_count_original'], bins=50, alpha=0.7, label='Original', color='#3498db')
axes[0, 1].hist(df_processed['word_count_processed'], bins=50, alpha=0.7, label='Procesado', color='#e74c3c')
axes[0, 1].set_xlabel('Número de palabras')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Número de Palabras', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Boxplot comparativo
df_box = pd.melt(df_processed, 
                 value_vars=['text_length_original', 'text_length_processed'],
                 var_name='Tipo', value_name='Longitud')
df_box['Tipo'] = df_box['Tipo'].replace({'text_length_original': 'Original', 'text_length_processed': 'Procesado'})
sns.boxplot(data=df_box, x='Tipo', y='Longitud', ax=axes[1, 0])
axes[1, 0].set_title('Comparación de Longitud', fontweight='bold')
axes[1, 0].grid(alpha=0.3, axis='y')

# Scatter plot: original vs procesado
axes[1, 1].scatter(df_processed['text_length_original'], df_processed['text_length_processed'], 
                   alpha=0.5, s=10, color='#2ecc71')
axes[1, 1].plot([0, df_processed['text_length_original'].max()], 
                [0, df_processed['text_length_original'].max()], 
                'r--', linewidth=2, label='Línea de igualdad')
axes[1, 1].set_xlabel('Longitud Original')
axes[1, 1].set_ylabel('Longitud Procesada')
axes[1, 1].set_title('Relación Original vs Procesado', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Guardar dataset preprocesado
output_path = Path('../data/processed/youtoxic_english_1000_processed.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Guardar solo columnas necesarias
columns_to_save = ['CommentId', 'VideoId', text_col, 'Text_processed', label_col]
if 'IsAbusive' in df_processed.columns:
    columns_to_save.append('IsAbusive')
if 'IsHatespeech' in df_processed.columns:
    columns_to_save.append('IsHatespeech')

df_processed[columns_to_save].to_csv(output_path, index=False)

print("="*80)
print("DATOS GUARDADOS")
print("="*80)
print(f"✅ Dataset preprocesado guardado en: {output_path}")
print(f"   Filas: {len(df_processed)}")
print(f"   Columnas: {len(columns_to_save)}")
print(f"\n📋 Columnas guardadas: {columns_to_save}")

# Mostrar muestra
print(f"\n📋 Muestra de datos guardados:")
df_processed[columns_to_save].head()


## 10. Resumen del preprocesamiento


In [ ]:
print("="*80)
print("RESUMEN DEL PREPROCESAMIENTO")
print("="*80)

print(f"\n✅ Pipeline aplicado:")
print(f"   1. Limpieza básica (URLs, emails, caracteres especiales)")
print(f"   2. Normalización (contracciones, repeticiones)")
print(f"   3. Tokenización ({'spaCy' if preprocessor.use_spacy else 'NLTK'})")
print(f"   4. Eliminación de stopwords")
print(f"   5. Lematización")

print(f"\n📊 Resultados:")
print(f"   - Textos procesados: {len(df_processed)}")
print(f"   - Reducción promedio de caracteres: {((df_processed['text_length_original'].mean() - df_processed['text_length_processed'].mean()) / df_processed['text_length_original'].mean() * 100):.1f}%")
print(f"   - Reducción promedio de palabras: {((df_processed['word_count_original'].mean() - df_processed['word_count_processed'].mean()) / df_processed['word_count_original'].mean() * 100):.1f}%")

print(f"\n💾 Archivo guardado:")
print(f"   {output_path}")

print("\n✅ Preprocesamiento completado exitosamente")
